<a href="https://colab.research.google.com/github/bkn872/computer_vision/blob/main/week2_cv_aiml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dog breed classification

### Mount google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


### Give project path

In [2]:
project_path = "/content/drive/My Drive/W2_Mentor/"

### Extract train.zip file

In [ ]:
from zipfile import ZipFile
with ZipFile(project_path+'train.zip', 'r') as z:
  z.extractall()

### Extract test.zip file

In [ ]:
from zipfile import ZipFile
with ZipFile(project_path+'test.zip', 'r') as z:
  z.extractall()

### Extract sample_submission.csv.zip file

In [ ]:
from zipfile import ZipFile
with ZipFile(project_path+'sample_submission.csv.zip', 'r') as z:
  z.extractall()

### Extract labels.csv.zip file

In [ ]:
from zipfile import ZipFile
with ZipFile(project_path+'labels.csv.zip', 'r') as z:
  z.extractall()

### Import pandas

In [ ]:
import pandas as pd

### Read labels.csv file

In [ ]:
labels = pd.read_csv('./labels.csv')
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


### Get breed counts

In [ ]:
breed_count = labels['breed'].value_counts()
breed_count.head()

scottish_deerhound      126
maltese_dog             117
afghan_hound            116
entlebucher             115
bernese_mountain_dog    114
Name: breed, dtype: int64

## One-hot encodings of labels

### Import numpy and encode labels['breed'] column

In [ ]:
import numpy as np
targets = pd.Series(labels['breed'])
one_hot = pd.get_dummies(targets, sparse = True)
one_hot_labels = np.asarray(one_hot)

In [ ]:
one_hot_labels[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

### You can also use label encoding if you want

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels['breed'])
print(integer_encoded)

[19 37 85 ...  3 75 28]


### Use keras.utils.to_categorical function

In [ ]:
import keras
y = keras.utils.to_categorical(integer_encoded, num_classes=None, dtype='int')

Using TensorFlow backend.


In [ ]:
y[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### Define img_rows, img_cols, and num_channel variables

In [ ]:
img_rows=128
img_cols=128
num_channel=1

### Get features

In [ ]:
from tqdm import tqdm
import cv2
x_feature = []
y_feature = []

i = 0 # initialisation
for f, img in tqdm(labels.values): # f for format ,jpg
    train_img = cv2.imread('./train/{}.jpg'.format(f),0)
    label = one_hot_labels[i]
    train_img_resize = cv2.resize(train_img, (img_rows, img_cols)) 
    x_feature.append(train_img_resize)
    y_feature.append(label)
    i += 1

100%|██████████| 10222/10222 [00:13<00:00, 781.85it/s]


### Normalize values

In [ ]:
x_train_data = np.array(x_feature, np.float32) / 255.   # /= 255 for normalisation
print (x_train_data.shape)
x_train_data = np.expand_dims(x_train_data, axis = 3) # for keras to given input to Conv2D layer
print (x_train_data.shape)

(10222, 128, 128)
(10222, 128, 128, 1)


In [ ]:
y_train_data = np.array(y_feature)
y_train_data.shape

(10222, 120)

### Split the training and validation data here

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train_data, y_train_data, test_size=0.2, random_state=2)
print (x_train.shape)
print (x_val.shape)

(8177, 128, 128, 1)
(2045, 128, 128, 1)


## Import keras and the required layers and model

In [ ]:
# import tensorflow as tf

from keras.models import Sequential  # initial NN
from keras.layers import Dense, Dropout # construct each layer
from keras.layers import Conv2D # swipe across the image by 1
from keras.layers import MaxPooling2D # swipe across by pool size
from keras.layers import Flatten, GlobalAveragePooling2D

### Define model and add layers

In [ ]:
model = Sequential()

model.add(Conv2D(32, (5,5), activation ='relu', input_shape = (img_rows, img_cols, num_channel))) 
model.add(MaxPooling2D(pool_size=3))
#model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Flatten()) 
# fully connected layer 
model.add(Dense(units = 500, activation = 'relu')) 
model.add(Dropout(0.2))
# output layer
model.add(Dense(units = 120, activation = 'softmax')) 






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


### Compile and summarize the model

In [ ]:
from keras.optimizers import Adam

optimizer = Adam(lr=0.01)
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"]) 
model.summary()



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 124, 124, 32)      832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 41, 41, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 41, 41, 64)        32832     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 20, 20, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 20, 20, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 10, 10, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 10, 128)    

### Define hyperparametrs and fit the model

In [ ]:
batch_size = 128
nb_epochs = 20
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=nb_epochs, 
                    validation_data=(x_val, y_val),
                    initial_epoch=0)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 8177 samples, validate on 2045 samples
Epoch 1/20





8177/8177 [==============================] - 20s 2ms/step - loss: 4.9037 - acc: 0.0092 - val_loss: 4.7870 - val_acc: 0.0078
Epoch 2/20
8177/8177 [==============================] - 5s 617us/step - loss: 4.7798 - acc: 0.0104 - val_loss: 4.7887 - val_acc: 0.0117
Epoch 3/20
8177/8177 [==============================] - 5s 619us/step - loss: 4.7786 - acc: 0.0125 - val_loss: 4.7909 - val_acc: 0.0117
Epoch 4/20
8177/8177 [==============================] - 5s 620us/step - loss: 4.7779 - acc: 0.0119 - val_loss: 4.7911 - val_acc: 0.0078
Epoch 5/20
8177/8177 [==============================] - 5s 624us/step - loss: 4.7768 - acc: 0.0097 - val_loss: 4.7918 - val_acc: 0.0117
Epoch 6/20
8177/8177 [==============================] - 5s 617us/step - loss: 4.7771 - acc: 0.0125 - val_loss: 4.7930 - val_acc: 0.0117
Epoch 7/20
8177/8177 [=============

## Using Transfer Learning

Download the vgg wieght file from here : https://github.com/MinerKasch/applied_deep_learning/blob/master/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

In [ ]:
from keras.applications.vgg16 import VGG16, preprocess_input

# Instantiate the model with the pre-trained weights (no top)
base_model= VGG16(weights=(project_path+'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'),
                 include_top=False, pooling='avg')

In [ ]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [ ]:
from tqdm import tqdm
import cv2
x_feature = []
y_feature = []

i = 0 # initialisation
for f, img in tqdm(labels.values): # f for format ,jpg
    train_img = cv2.imread('./train/{}.jpg'.format(f),1)
    label = one_hot_labels[i]
    train_img_resize = cv2.resize(train_img, (img_rows, img_cols)) 
    x_feature.append(train_img_resize)
    y_feature.append(label)
    i += 1

100%|██████████| 10222/10222 [00:31<00:00, 327.36it/s]


### Get Image embedding from the pre-trained model and use that as input to present model

In [ ]:
from keras.preprocessing import image

X_arr = []
# X_arr = base_model.predict(preprocess_input(img_to_array(image.load_img('./train/'+labels['id'][0]+'.jpg', target_size=[128,128]))))

for id, label in tqdm(labels.values):
    X_arr.append(base_model.predict(preprocess_input(np.expand_dims(image.img_to_array(image.load_img('./train/'+id+'.jpg', target_size=[128,128])), axis=0)))[0])
X = pd.DataFrame(X_arr)

100%|██████████| 10222/10222 [01:38<00:00, 103.64it/s]


In [ ]:
from keras.layers import Activation
model = Sequential([
    Dense(1024, input_shape=(512,)),
    Activation('relu'),
    Dense(256, input_shape=(512,)),
    Activation('relu'),
    Dense(120),
    Activation('softmax'),
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
X_train = X
y_train = y_feature

In [ ]:
model.fit(X_train, np.asarray(y_train), epochs=100, batch_size=100, verbose=True)

Epoch 1/100
10222/10222 [==============================] - 1s 82us/step - loss: 4.7270 - acc: 0.1511
Epoch 2/100
10222/10222 [==============================] - 1s 56us/step - loss: 2.2170 - acc: 0.4311
Epoch 3/100
10222/10222 [==============================] - 1s 54us/step - loss: 1.5203 - acc: 0.5806
Epoch 4/100
10222/10222 [==============================] - 1s 57us/step - loss: 1.1159 - acc: 0.6781
Epoch 5/100
10222/10222 [==============================] - 1s 58us/step - loss: 0.7727 - acc: 0.7750
Epoch 6/100
10222/10222 [==============================] - 1s 58us/step - loss: 0.4882 - acc: 0.8624
Epoch 7/100
10222/10222 [==============================] - 1s 59us/step - loss: 0.2953 - acc: 0.9204
Epoch 8/100
10222/10222 [==============================] - 1s 52us/step - loss: 0.1505 - acc: 0.9667
Epoch 9/100
10222/10222 [==============================] - 1s 52us/step - loss: 0.1104 - acc: 0.9786
Epoch 10/100
10222/10222 [==============================] - 1s 53us/step - loss: 0.0989 - a